In [ ]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

rc('animation', html='jshtml')
rc('savefig', dpi=100)
matplotlib.rcParams["savefig.format"] = 'svg'

## Define dataset

In [ ]:
# #Define a simple range of values for x and reshape so sklearn likes it.
np.random.seed(127)
x = np.linspace(0,1,200)
noise = np.random.normal(0,0.02, x.shape)
x = x + noise
x = np.sort(x)

# y= (2 * np.sin(10*x) + np.cos(20 * x - 3) + 3 * np.log(10*x + 0.5) - 4)/6.
y = np.cos(x)+0.8*x -0.5 
noise = np.random.normal(0,0.03, x.shape)
y = y + noise
x = x - 0.5

%matplotlib inline
plt.scatter(x, y, marker='.', color='g')
plt.xlabel("X(input)")
plt.ylabel("y(output)")
plt.show()
# plt.savefig("data_set.svg")

## Least Squares Method of Linear Regression

In [ ]:
def regression(x,y):
    n = len(x)
    sy = np.sum(y)
    sx = np.sum(x)
    sx2 = np.sum(np.square(x))
    sxy = np.sum(x*y)

    # y = mx+c
    z = (n*sx2 - sx**2)
    m = (n*sxy - sx*sy)/z
#     c = (sy*sx2 - sx*sxy)/z
    c = (sy - m*sx)/n
    return m, c

def output(x, m, c):
    return m*x+c

def error(target, output): ### Root mean squared Error
    return np.sqrt(np.square(target-output).mean())

def error1(target, output): ### Mean Squared error
    return np.square(target-output).mean()

def error2(target, output): ### MODIFIED Mean Squared error
    return np.square(target-output).mean()/2

In [ ]:
m, c = regression(x, y)
y_hat = output(x,m,c)
plt.scatter(x, y, marker='.', color='g', label='data')
plt.xlabel("X(input)")
plt.ylabel("y(output)")

plt.scatter(x, y_hat, marker='.', color='r', label='prediction')
plt.legend()
plt.show()

In [ ]:
m, c

In [ ]:
min_err = error1(y, y_hat)
min_err

In [ ]:
error_analytical = error2(y, y_hat)
error_analytical

In [ ]:
x_ = 0.1
y_ = output(x_, m, c)

plt.scatter(x, y, marker='.', color='g', label='data')
plt.xlabel("X(input)")
plt.ylabel("y(output)")

plt.scatter(x, y_hat, marker='.', color='r', label='prediction')

plt.scatter(x_, y_, marker='o', color='b', lw=4, label='test')

plt.legend()
plt.show()

## Linear Regression using Linear Algebra

In [ ]:
X = np.c_[x, np.ones(len(x))]
X

In [ ]:
X.shape

In [ ]:
Y = y.reshape(-1,1) 
Y

In [ ]:
Y.shape

In [ ]:
W = np.linalg.pinv(X) @ Y
W

In [ ]:
W.shape

In [ ]:
y_hat = X @ W
X_ = np.array([[0.1, 1]])
Y_ = X_ @ W

plt.scatter(x, y, marker='.', color='g', label='data')
plt.xlabel("X(input)")
plt.ylabel("y(output)")

plt.scatter(x, y_hat, marker='.', color='r', label='prediction')

plt.scatter(X_[0,0], Y_, marker='o', color='b', lw=4, label='test')

plt.legend()
plt.show()

## Linear Regression using Gradient descent

#### Random Search

In [ ]:
y.min(), y.max()

In [ ]:
total = 100000
min_err = 999999
m_search = None
c_search = None
np.random.seed(127)
y_hat_search = None
for count in range(total):
    theta = np.random.uniform(low=0, high=np.pi)
    m_ = np.tan(theta)
    c_ = np.random.uniform(low=0.425, high=0.909)
    y_hat = output(x, m_, c_)
    err = error1(y, y_hat)
    if err < min_err:
        min_err = err
        m_search = m_
        c_search = c_
        y_hat_search = y_hat
m_search, c_search

In [ ]:
min_err

In [ ]:
plt.scatter(x, y, marker='.', color='g', label='data')
plt.xlabel("X(input)")
plt.ylabel("y(output)")

plt.scatter(x, y_hat_search, marker='.', color='r', label='prediction')
plt.legend()
plt.show()

### Gradient Descent Search

In [ ]:
E = 1e10
Eprev = None

In [ ]:
np.random.seed(129)
m = np.random.normal(loc=1)
c = np.random.uniform()
m,c

In [ ]:
n = len(x)
alpha = 0.1
epsilon = 1e-10

val_list = []

In [ ]:
for step in range(10000000):
    #### Set Previous error
    Eprev = E    
    #### calculating output
    y_hat = output(x, m, c)
    #### calculating error
    E = error2(y, y_hat)

    #### calculating gradients
    dy = y_hat - y
    dm = (dy*x).sum()/n
    dc = dy.sum()/n

    m = m - alpha*dm
    c = c - alpha*dc


    if step%10 == 0:
        print("step = ",step)
        print("Error = ", E)
        
        val_list.append((E,m,c,step))

        if Eprev-E < epsilon:
            print('Optimized to our threshold')
            break

In [ ]:
emcs = np.array(val_list)
emcs.shape

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.close()

# animation function. This is called sequentially  
def animate(i):
    emcs_i = emcs[i]
    E, m, c, step = emcs_i.tolist()
    step = int(step)
    
    y_hat = output(x, m, c)    
    
    ax.clear()
    ax.set_xlim((-0.5679472377938085, 0.5638641593022863))
    ax.set_ylim((0.39317623114955275, 0.9407836786747326))

    ax.scatter(x, y, marker='.', color='g', label='data')
    ax.scatter(x, y_hat, marker='.', color='r', label='prediction')
    ax.text(-0.43,0.97,f'Step={step}, E={E:.6f} --> m={m:.5f} c={c:.5f}')


anim = animation.FuncAnimation(fig, animate,
                             frames=len(emcs), interval=100)

rc('animation', html='jshtml')
anim

In [ ]:
m,c

In [ ]:
y_hat = output(x, m, c)
#### calculating mean squared error
error1(y, y_hat), error2(y, y_hat)

### Plotting the error surface and optimization trajectory

In [ ]:
num_points = 100
mgM = np.linspace(-0.8, 1.2, num_points)  ## for mesh grid
mgC = np.linspace(0.4, 1, num_points)

In [ ]:
mgM.shape , mgC.shape

In [ ]:
mgM, mgC = np.meshgrid(mgM, mgC)

In [ ]:
mgE = np.array([error2(y, output(x, mgm, mgc)) \
                for mgm, mgc in zip(np.ravel(mgM), np.ravel(mgC))]).reshape(mgM.shape)

In [ ]:
mgE.shape

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
# matplotlib.rcParams['figure.figsize'] = (12, 8)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(mgM, mgC, mgE)
ax.set_xlabel('m')
ax.set_ylabel('c')
ax.set_zlabel('E')

plt.show()

In [ ]:
%matplotlib inline
# ratio = 25/18
# width, height = int(8*ratio), int(6*ratio)
fig = plt.figure()#figsize=(8,6))
ax = fig.gca(projection='3d')
plt.close()

#set initial viewing angles
azimuth, elev = -36, 41
ax.view_init(elev, azimuth )

# animation function. This is called sequentially  
def animate(step):
    if step%10 == 0:
        print(f"Step {step}")
    E_, m_, c_, iterations = emcs[step].tolist()

    ax.clear()
    ax.plot_surface(mgM, mgC, mgE, cmap='hot', alpha=0.7)
    ax.scatter(emcs[step,1],emcs[step,2], emcs[step,0], c='g', lw=3)
    ax.plot(xs=emcs[:step,1], ys=emcs[:step,2], zs=emcs[:step,0],c='orange', lw=2)
    ax.set_xlabel('m')
    ax.set_ylabel('c')
    ax.set_zlabel('E')

    iterations = int(iterations)
    ax.text(-0.85,0.4,0.15,f'Step={iterations}, E={E_:.6f} --> m={m_:.5f} c={c_:.5f}')


anim = animation.FuncAnimation(fig, animate, init_func=lambda:None,
                             frames=len(emcs), interval=100)

In [ ]:
anim

In [ ]:

HTML(anim.to_html5_video())

In [ ]:
writer_gif = animation.ImageMagickWriter(fps=10)
anim.save('error_surface_gd_1.gif', writer=writer_gif, dpi=100)

In [ ]:
errs = val_mat[:,0]

In [ ]:
%matplotlib inline
plt.xlabel("Steps")
plt.ylabel("Error(modified)")
plt.plot([0, len(errs)], [error_analytical, error_analytical], label="Error (analytical)")
plt.plot(errs, label="Error (gradient descent)")
plt.legend()

# plt.savefig("error_plot_step.svg")

In [ ]:
plt.plot(val_mat[:,1]) ## This is the slope value wrt steps

In [ ]:
plt.plot(val_mat[:,2]) ## This is the y-intercept value wrt steps

In [ ]:
###### This is for feature image

matplotlib.rcParams['figure.figsize'] = (16, 8)
m, c = regression(x, y)
y_hat = output(x,m,c)
plt.scatter(x, y, marker='o', color='g', label='data', alpha=0.5)
plt.xlabel("X(input)")
plt.ylabel("y(output)")

# plt.plot(x, y_hat, marker='o', color='r', label='prediction', alpha=0.5)
plt.plot(x, y_hat, color='r', lw=5, label='prediction', alpha=0.5)
plt.legend()
# plt.savefig("regression_feature.svg")